In [1]:
import mhkit
from mhkit.wave import resource, performance, graphics
from sklearn.mixture import GaussianMixture
from mhkit.wave.io import ndbc
import matplotlib.pyplot as plt
from matplotlib import colors 
from scipy import stats
import pandas as pd
import numpy as np
import calendar

In [2]:
import geopandas

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
parameter = 'swden'
buoy_number = '46050' 
ndbc_available_data= ndbc.available_data(parameter, buoy_number)
years_of_interest = ndbc_available_data[ndbc_available_data.year < 2013]
filenames= years_of_interest['filename']
ndbc_requested_data = ndbc.request_data(parameter, filenames)

In [ ]:
ndbc_data={}
# Create a Datetime Index and remove NOAA date columns for each year
for year in ndbc_requested_data:
    year_data = ndbc_requested_data[year]
    ndbc_data[year] = ndbc.to_datetime_index(parameter, year_data)

In [ ]:
res = resource.energy_flux(ndbc_data['1996'].T,h=399.)

In [ ]:
# Intialize empty lists to store the results from each year
Hm0_list=[]
Te_list=[]
J_list=[]
Tp_list=[]

# Iterate over each year and save the result in the initalized dictionary
for year in ndbc_data:
    year_data = ndbc_data[year]
    Hm0_list.append(resource.significant_wave_height(year_data.T))
    Te_list.append(resource.energy_period(year_data.T))
    J_list.append(resource.energy_flux(year_data.T, h=399.))
    Tp_list.append(resource.peak_period(year_data.T))

# Concatenate list of Series into a single DataFrame
Te = pd.concat(Te_list ,axis=0)
Tp = pd.concat(Tp_list ,axis=0)
Hm0 = pd.concat(Hm0_list ,axis=0)
J = pd.concat(J_list ,axis=0)
data = pd.concat([Hm0, Te, Tp, J],axis=1)

# Drop any NaNs created from the calculation of Hm0 or Te
data.dropna(inplace=True)
# Sort the DateTime index
data.sort_index(inplace=True)
data

## Annual Scatter Table

In [ ]:
# Remove Hm0 Outliers
data_clean = data[data.Hm0 < 20]
sigma = data_clean.J.std()
data_clean = data_clean[data_clean.J > (data_clean.J.mean() - 0.9* sigma)]
#data_clean = data_clean[data_clean.J < data_clean.J.mean() + 3* data_clean.J.std()]

Hm0_bin_size = 0.5
Hm0_edges = np.arange(0,15+Hm0_bin_size,Hm0_bin_size)


Te_bin_size = 1
Te_edges = np.arange(0, 20+Te_bin_size,Te_bin_size)
Te_edges

h= plt.hist2d(data_clean.Te,  
              data_clean.Hm0, 
              bins = (Te_edges,Hm0_edges), 
              norm = colors.LogNorm())

In [ ]:
data.describe()

## Wave Power by Month

In [ ]:
months=data_clean.index.month
data_group=data_clean.groupby(months)
# 3 plots
QoIs = data_clean.keys()
fig, axs = plt.subplots(len(QoIs),1, figsize=(8, 6), sharex=True)
#shade between 25% and 75%
QoIs = data_clean.keys()
for i in range(len(QoIs)):
    QoI = QoIs[i]
    axs[i].plot(data_group.median()[QoI], marker='.')

    axs[i].fill_between(months.unique(),
                        data_group.describe()[QoI,   '25%'],
                        data_group.describe()[QoI,   '75%'],
                        alpha=0.2)

plt.setp(axs[3], xlabel='Month')
plt.setp(axs[0], ylabel='Hm0 [m]')

plt.setp(axs[1], ylabel='Te  [s]')
plt.setp(axs[2], ylabel='J   [kW/m]')
plt.setp(axs[3], ylabel='Tp  [s]')

#significant steepness


plt.tight_layout()

In [ ]:
cumSum={}
for month in data_clean.index.month.unique():    
    F = mhkit.river.resource.exceedance_probability(data_clean[data_clean.index.month==month].J)
    cumSum[month] = 1-F/100
    cumSum[month].sort_values('F', inplace=True)
plt.figure(figsize=(12,8) )
for month in data_clean.index.month.unique():
    plt.semilogx(data_clean.loc[cumSum[month].index].J, cumSum[month].F, '--', label=calendar.month_abbr[month])

F = mhkit.river.resource.exceedance_probability(data_clean.J)
F.sort_values('F', inplace=True)
plt.semilogx(data_clean.loc[F.index].J, 1-F/100, 'k-', fillstyle='none', label='All')

plt.xlim([1000, 1E6])    
plt.grid()
plt.xlabel('Energy Flux')
plt.ylabel('Cumulative Distribution')
plt.legend()

In [ ]:
ndbc_data['1996'] = ndbc_data['1996'][ndbc_data['1996'] != 999.0].dropna()
ndbc_data['1996'].to_csv('46050_1996.csv')
ndbc_data['1996'].head()

In [ ]:
# Compute Gaussian Mixture Model for each number of clusters
Ns= [4, 8, 16, 32, 64]
X = np.vstack((data_clean.Te.values, data_clean.Hm0.values)).T
fig, axs = plt.subplots(len(Ns),1, figsize=(8, 24), sharex=True)

results={}
for N in Ns:
    gmm = GaussianMixture(n_components=N).fit(X)

    # Save centers and weights
    result = pd.DataFrame(gmm.means_, columns=['Te','Hm0'])
    result['weights'] = gmm.weights_

    result['Tp'] = result.Te / 0.858
    results[N] = result
    
    
    labels = gmm.predict(X)
    
    i = Ns.index(N)
    axs[i].scatter(data_clean.Te.values, data_clean.Hm0.values, c=labels, s=40)
    axs[i].plot(result.Te, result.Hm0, 'm+')
    axs[i].title.set_text(f'{N} Clusters')
    plt.setp(axs[i], ylabel='Energy Period, $T_e$ [s]')
plt.setp(axs[len(Ns)-1], xlabel='Sig. wave height, $Hm0$ [m')    

## Calculate Energy Flux

Calculate energy flux for each cluster

In [ ]:
w = ndbc_data[year].columns.values
f = w / 2*np.pi


for N in results:
    result = results[N]
    J=[]
    for i in range(len(result)):
        b = resource.bretschneider_spectrum(f, result.Tp[i], result.Hm0[i])
        J.extend([resource.energy_flux(b, h=399.).values[0][0]])
    
    result['J']  = J
    results[N] = result

In [ ]:
Hm0_max=14
Te_max=16

reduce_factor=4

Hm0_bin_size = 0.5/reduce_factor
Te_bin_size = 1.0/reduce_factor

Hm0_bins = np.arange(0, Hm0_max + Hm0_bin_size, Hm0_bin_size)    
Te_bins = np.arange(0, Te_max + Te_bin_size, Te_bin_size)
#JM = mhkit.wave.performance.wave_energy_flux_matrix(data_clean.Hm0, data_clean.Te, data_clean.J, 'mean', Hm0_bins, Te_bins)
JM = performance.wave_energy_flux_matrix(data_clean.Hm0, data_clean.Te, data_clean.J, 'mean', Hm0_bins, Te_bins)


fig, ax = plt.subplots(figsize=(20, 10))
ax = graphics.plot_matrix(JM, ax=ax, show_values=False)
#L = mhkit.wave.performance.capture_length(P, data_clean.J) 

#maep_timeseries = mhkit.wave.performance.mean_annual_energy_production_timeseries(L, data_clean.J)
#print("MAEP from timeseries = ", maep_timeseries)

In [ ]:
Jmean, xe, ye, bn = stats.binned_statistic_2d(data_clean.Hm0, data_clean.Te, data_clean.J,
                                                  statistic='sum',bins=[Te_bins, Hm0_bins])
#H, xe,ye = np.histogram2d(data_clean.Hm0, data_clean.Te, bins=[Te_bins, Hm0_bins], density=True)

# Incident  Wave PowerResults

## Full Sea State


In [ ]:
nHours = (data_clean.index[1] - data_clean.index[0]).seconds/3600
Total = data_clean.J.sum() * nHours
print(f'{Total} (W*hr)/m')

## 2D Histogram Result

In [ ]:
x = Jmean.flatten() #* H.flatten()
x = x[~np.isnan(x)]
x.sum()/Total

## K-means

In [ ]:
ratios={}
for N in results:
    ratios[N] = np.round((results[N].J*len(data_clean)* results[N].weights).sum()/Total,4)
    
pd.Series(ratios)

In [ ]:
#MAP OF THE AREA
